# Writing out a USGSCSM ISD from a PDS3 Kaguya image

In [1]:
import pvl
import numpy as np
import json
import os
import pandas as pd
import knoten
import csmapi
import ale
from ale.drivers.kaguya_drivers import KaguyaTcPds3NaifSpiceDriver
from ale.formatters.usgscsm_formatter import to_usgscsm

os.environ['ISISROOT'] = '/usgs/pkgs/isis3.8.0_RC1/install'
from pysis import isis
from pysis.exceptions import ProcessError

In [2]:
fileName = '/home/arsanders/testData/TC1S2B0_01_02842S506E1942.img'

model="USGS_ASTRO_LINE_SCANNER_SENSOR_MODEL"
plugin = csmapi.Plugin.getList()[0]
isd = csmapi.Isd(fileName)

warns = csmapi.WarningList()
if plugin.canModelBeConstructedFromISD(isd, model, warns):
    camera = plugin.constructModelFromISD(isd, model)
else:
    print("Unable to construct camera model.")
    for item in warns:
        print(item.getMessage())

In [3]:
# Set the output location of the resulting .cub
cub_loc = '/home/arsanders/usgscsm/tests/data/TC1S2B0_01_02842S506E1942.cub'

try: 
    isis.kaguyatc2isis(from_=fileName, to=cub_loc)
except ProcessError as e:
    print(e.stderr)

In [4]:
try:
    isis.spiceinit(from_=cub_loc, shape='ellipsoid')
except ProcessError as e:
    print(e.stderr)

In [13]:
def check_pixel(camera, cub, line, sample):
    output = isis.campt(from_=cub, line=line, sample=sample)
    pvl_output = pvl.loads(output)
    bodyfixed = pvl_output['GroundPoint']['BodyFixedCoordinate']
    bodyfixed = np.asarray(bodyfixed.value) * 1000
    image_coord = camera.groundToImage(csmapi.EcefCoord(*bodyfixed))
    line_diff = pvl_output['GroundPoint']['Line'] - image_coord.line - .5
    sample_diff = pvl_output['GroundPoint']['Sample'] - image_coord.samp - .5
    return (line_diff, sample_diff)

In [14]:
isis_label = pvl.load(cub_loc)
n_samples = isis_label['IsisCube']['Core']['Dimensions']['Samples']
n_lines = isis_label['IsisCube']['Core']['Dimensions']['Lines']

In [15]:
pixels_dict = {'line' : [1,1,n_lines, n_lines, n_lines/2],
               'sample' : [1, n_samples, 1, n_samples, n_samples/2]}

pixels_df = pd.DataFrame.from_dict(pixels_dict)
pixels_df['line_diff'] = np.NaN
pixels_df['sample_diff'] = np.NaN

for _, row in pixels_df.iterrows():
    row['line_diff'], row['sample_diff'] = check_pixel(camera, cub_loc, row['line'], row['sample'])

In [16]:
pixels_df

,line,sample,line_diff,sample_diff
0,1.0,1.0,1.110291,1.579313
1,1.0,1744.0,1.110945,1.492453
2,4656.0,1.0,-1.048140,1.599806
3,4656.0,1744.0,-1.047574,1.463186
4,2328.0,872.0,-1.042982,1.532304
